<a href="https://colab.research.google.com/github/meabhl85/Canvas-System/blob/main/CAN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Imports and Google Drive Connection

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import backend
import numpy as np
from numpy.random import randn
from numpy.random import randint
from numpy import expand_dims
from numpy import mean
from numpy import ones
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt
from matplotlib import pyplot
from tensorflow.keras.initializers import RandomNormal

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

#Nicely formatted time string
def hms_string(sec_elapsed):
  h = int(sec_elapsed / (60 * 60))
  m = int((sec_elapsed % (60 * 60)) / 60)
  s = sec_elapsed % 60
  return "{}:{:>02}:{:>05.2f}".format(h, m, s)

Mounted at /content/drive
Note: using Google CoLab


#### Model Hyperparameters

In [3]:
#Image Sizes
GENERATE_RES = 5 # Generation resolution factor 
GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
IMAGE_CHANNELS = 3

#Input and Output Paths
NPY_FILES_PATH = '/content/drive/MyDrive/Colab Notebooks/honours_project/Npy_Files/'
OUTPUT_IMAGE_PATH = '/content/drive/MyDrive/Colab Notebooks/honours_project/GAN_Images/'

# Preview image 
PREVIEW_ROWS = 2
PREVIEW_COLS = 3
PREVIEW_MARGIN = 16

# Size vector to generate images from
SEED_SIZE = 800
EPOCHS = 1000
BATCH_SIZE = 32
BUFFER_SIZE = 60000

#### Model Class

In [4]:
class CAN:
  def __init__(self):
    self.class_count = 0
    class_names = []
    init = True
    self.image_shape = (GENERATE_SQUARE, GENERATE_SQUARE, IMAGE_CHANNELS)
    #Define cross_entropy function
    self.cross_entropy = tf.keras.losses.BinaryCrossentropy()

    for filename in tqdm(os.listdir(NPY_FILES_PATH)):
      npy_file = os.path.join(NPY_FILES_PATH, filename)
      class_names.append(filename)
      AC_training_data = np.load(npy_file)
      AC_labels = [self.class_count] * len(AC_training_data)
      AC_labels = np.array(AC_labels)

      if init == True:
        self.training_data = AC_training_data
        self.training_labels = AC_labels
        init == False
      else:  
        #Combine files
        self.training_data = np.concatenate([self.training_data, AC_training_data])
        self.training_labels = np.concatenate([self.training_labels, AC_labels])
      break
    self.training_dataset = tf.data.Dataset.from_tensor_slices(self.training_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)  
    self.training_labels = tf.data.Dataset.from_tensor_slices(self.training_labels).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    self.class_count += 1


  def build_generator(self):
    model = Sequential()

    model.add(Dense(4*4*256, activation = "relu", input_dim = SEED_SIZE))
    model.add(Reshape((4, 4, 256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size = 3, padding = "same"))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size = 3, padding = "same"))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Activation("relu"))
   
    # Output resolution, additional upsampling
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size = 3, padding = "same"))
    model.add(BatchNormalization(momentum = 0.8))
    model.add(Activation("relu"))

    if GENERATE_RES > 1:
      model.add(UpSampling2D(size = (GENERATE_RES, GENERATE_RES)))
      model.add(Conv2D(128,kernel_size=3, padding="same"))
      model.add(BatchNormalization(momentum = 0.8))
      model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv2D(IMAGE_CHANNELS, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    return model

  def build_discriminator(self):
    
    init = RandomNormal(stddev=0.02)
    inputs = tf.keras.Input(shape=self.image_shape)

    x = tf.random.normal(self.image_shape)

    #Layer 1
    # layer_1 = tf.keras.layers.Conv2D(32, kernel_size = 3, strides = 2, padding = "same", input_shape=image_shape[1:])(x)
    layer_1 = tf.keras.layers.Conv2D(32, kernel_size = 3, kernel_initializer=init, strides = 2, padding = "same")(inputs)
    relu_1 = tf.keras.layers.LeakyReLU(alpha = 0.2)(layer_1)

    #Layer 2
    dropout_1 = tf.keras.layers.Dropout(0.25)(relu_1)
    layer_2 = tf.keras.layers.Conv2D(64, kernel_size = 3, kernel_initializer=init, strides = 2, padding = "same")(dropout_1)
    zero_padding_1 = tf.keras.layers.ZeroPadding2D(padding = ((0, 1), (0, 1)))(layer_2)
    batch_norm_1 = tf.keras.layers.BatchNormalization(momentum = 0.8)(zero_padding_1)
    relu_2 = tf.keras.layers.LeakyReLU(alpha = 0.2)(batch_norm_1)

    #Layer 3
    dropout_2 = tf.keras.layers.Dropout(0.25)(relu_2)
    layer_3 = tf.keras.layers.Conv2D(128, kernel_size = 3, kernel_initializer=init, strides = 2, padding = "same")(dropout_2)
    batch_norm_2 = tf.keras.layers.BatchNormalization(momentum = 0.8)(layer_3)
    relu_3 = tf.keras.layers.LeakyReLU(alpha = 0.2)(batch_norm_2)

    #Layer 4 
    dropout_3 = tf.keras.layers.Dropout(0.25)(relu_3)
    layer_4 = tf.keras.layers.Conv2D(256, kernel_size = 3, kernel_initializer=init, strides = 1, padding = "same")(dropout_3)
    batch_norm_3 = tf.keras.layers.BatchNormalization(momentum = 0.8)(layer_4)
    relu_4 = tf.keras.layers.LeakyReLU(alpha = 0.2)(batch_norm_3)

    #Layer 5
    dropout_4 = tf.keras.layers.Dropout(0.25)(relu_4)
    layer_5 = tf.keras.layers.Conv2D(512, kernel_size = 3, kernel_initializer=init, strides = 1, padding = "same")(dropout_4)
    batch_norm_4 = tf.keras.layers.BatchNormalization(momentum = 0.8)(layer_5)
    relu_5 = tf.keras.layers.LeakyReLU(alpha = 0.2)(batch_norm_4)

    #Real/Fake Output
    dropout_5 = tf.keras.layers.Dropout(0.25)(relu_5)
    flatten_1 = tf.keras.layers.Flatten()(dropout_5)
    discriminator_output = tf.keras.layers.Dense(1, kernel_initializer=init, name="RF_Output")(flatten_1)

    #Layer 7
    layer_7 = tf.keras.layers.Dense(1024, kernel_initializer=init, name = 'd_h8_lin')(relu_5)
    relu_7 = tf.keras.layers.LeakyReLU(alpha = 0.2)(layer_7)

    #Layer 8
    layer_8 = tf.keras.layers.Dense(512, kernel_initializer=init, name = 'd_h9_lin')(relu_7)
    relu_7 = tf.keras.layers.LeakyReLU(alpha = 0.2)(layer_8)
    
    flatten_2 = tf.keras.layers.Flatten()(relu_7)
    discriminator_class_output = tf.keras.layers.Dense(self.class_count, kernel_initializer=init, name = 'd_co_lin')(flatten_2)
    discriminator_class_output_softmax = tf.nn.softmax(discriminator_class_output)   

    # #Class Output
    # output_2 = tf.keras.layers.Dense(class_count, activation=tf.nn.softmax, name="Class_Output")(discriminator_class_output)
    model = tf.keras.Model(inputs = inputs, outputs = [tf.nn.sigmoid(discriminator_output), discriminator_output, discriminator_class_output, discriminator_class_output_softmax])

    return model

  def discriminator_rf_loss(self, disc_output_linear, disc_output_sigmoid):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = disc_output_linear, 
                                                                labels = tf.ones_like(disc_output_sigmoid)))

  def discriminator_class_loss(self, disc_class_output, training_class_output):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = disc_class_output, 
                                                                  labels = training_class_output))

  # def generator_class_loss(disc_class_output, disc_class_output_softmax):
  #   return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = disc_class_output,
                                                                  # labels = (1.0 / 27) * tf.ones_like(disc_class_output_softmax)))

  def generator_class_loss(self, disc_class_output, disc_class_output_softmax):
    # training_labels = tf.convert_to_tensor(, np.float32)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = disc_class_output,
                                                                  labels = (1.0 / self.class_count) * tf.ones_like(disc_class_output_softmax)))

  def discriminator_total_loss(self, real_disc_output_linear, real_disc_output_sigmoid, fake_disc_output_linear, fake_disc_output_sigmoid, real_image_labels, real_disc_class_output):
      discriminator_real_loss = discriminator_rf_loss(real_disc_output_linear, real_disc_output_sigmoid)
      discriminator_fake_loss = discriminator_rf_loss(fake_disc_output_linear, fake_disc_output_sigmoid)

      real_image_labels = tf.cast(real_image_labels, tf.float32)
      real_disc_class_output = tf.convert_to_tensor(real_disc_class_output, np.float32)

      discriminator_loss_class_real = discriminator_class_loss(real_disc_class_output, real_image_labels)

      discriminator_l = discriminator_real_loss + discriminator_fake_loss + discriminator_loss_class_real

      return discriminator_l

  def generator_loss(self, fake_pred):
    #As G wants to create real images the loss is cross entropy of 1's and the generated output
    return self.cross_entropy(tf.ones_like(fake_pred), fake_pred)

  def discriminator_loss(self, real_pred, fake_pred):
    #D should compare the real predictions to 1 and fake to 0 (as they are the real values)
    real_image_loss = self.cross_entropy(tf.ones_like(real_pred), real_pred)
    fake_image_loss = self.cross_entropy(tf.zeros_like(fake_pred), fake_pred)

    #Discriminator uses the complete loss
    final_loss = real_image_loss + fake_image_loss
    return final_loss

  def compile_models(self):
    self.discriminator_optimiser = tf.keras.optimizers.Adam(0.0002, 0.5)
    self.generator_optimiser = tf.keras.optimizers.Adam(0.0002, 0.5)

  # @tf.function
  def step(self, real_images, real_image_labels):
    seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])

    #Performs automatic differentiation used for backpropagation 
    with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
      #Generate images with noise vector
      generated_images = self.generator(seed, training = True)

      #Put resulting image into the D and return prediction
      real_disc_output_sigmoid, real_disc_output_linear, real_disc_class_output, real_disc_class_output_softmax = self.discriminator(real_images, training = True)
      real_disc_class_output = np.argmax(real_disc_class_output, 1)
      real_disc_class_output_softmax = np.argmax(real_disc_class_output_softmax, 1)

      fake_disc_output_sigmoid, fake_disc_output_linear, fake_disc_class_output, fake_disc_class_output_softmax = self.discriminator(generated_images, training = True)
      fake_disc_class_output = np.argmax(fake_disc_class_output, 1)
      fake_disc_class_output_softmax = np.argmax(fake_disc_class_output_softmax, 1)

      #Calculate loss for both D and G
      # g_loss = generator_loss(fake_pred)
      # d_loss = discriminator_loss(real_pred, fake_pred)

      #DISCRIMINATOR LOSS
      discriminator_real_loss = self.discriminator_rf_loss(real_disc_output_linear, real_disc_output_sigmoid)
      discriminator_fake_loss = self.discriminator_rf_loss(fake_disc_output_linear, fake_disc_output_sigmoid)

      real_image_labels = tf.cast(real_image_labels, tf.float32)
      real_disc_class_output = tf.convert_to_tensor(real_disc_class_output, np.float32)
      discriminator_loss_class_real = self.discriminator_class_loss(real_disc_class_output, real_image_labels)
      
      discriminator_loss = discriminator_real_loss + discriminator_fake_loss + discriminator_loss_class_real

      #GENERATOR LOSS
      a = tf.ones_like(fake_disc_class_output_softmax)
      fake_disc_class_output_softmax_temp = (1.0 / 27) * a.numpy()
      fake_disc_class_output_softmax_temp = tf.convert_to_tensor(fake_disc_class_output_softmax_temp, np.float32)
      fake_disc_class_output = tf.convert_to_tensor(fake_disc_class_output, np.float32)

      generator_loss_class_fake = self.generator_class_loss(fake_disc_class_output, fake_disc_class_output_softmax_temp)
      generator_loss_fake = self.discriminator_rf_loss(fake_disc_output_linear, fake_disc_output_sigmoid)

      generator_loss = generator_loss_fake + 1.0 * generator_loss_class_fake

    #Calculate gradients
    gen_gradients = g_tape.gradient(generator_loss, self.generator.trainable_variables)
    dis_gradients = d_tape.gradient(discriminator_loss, self.discriminator.trainable_variables)
    
    # Update weights 
    self.generator_optimiser.apply_gradients(zip(gen_gradients, self.generator.trainable_variables))
    self.discriminator_optimiser.apply_gradients(zip(dis_gradients, self.discriminator.trainable_variables))

    print(generator_loss)
    print(discriminator_loss)

    return generator_loss, discriminator_loss

  def train(self, training_dataset, training_labels, epochs, save_img_checkpoint, save_model_checkpoint):
    fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, SEED_SIZE))
    start = time.time()
    save_count = 0

    for epoch in range(epochs):
      #Initalise timing and loss lists
      epoch_time = time.time()
      g_loss_list = []
      d_loss_list = []

      #Run the step training
      for real_images, real_image_labels in zip(training_dataset, training_labels):
        t = self.step(real_images, real_image_labels)
        g_loss_list.append(t[0])
        d_loss_list.append(t[1])
      
      
      #Average losses for entire image batch
      average_g_loss = sum(g_loss_list) / len(g_loss_list)
      average_d_loss = sum(d_loss_list) / len(d_loss_list)

      #Calculate epoch time
      epoch_finished = time.time() - epoch_time
      print (f'Epoch {epoch + 1}, gen loss={average_g_loss}, disc loss={average_d_loss}, epoch time={hms_string(epoch_finished)}')
      
      if save_count == 10:
        self.save_images(epoch, fixed_seed)
        save_count = 0

      save_count += 1

    final_time = time.time()-start
    print (f'Training time: {hms_string(final_time)}')

  def save_models(self):
    output_model_path = os.path.join(OUTPUT_IMAGE_PATH, self.data_name + "_Models")
    if not os.path.exists(output_model_path):
      os.makedirs(output_model_path)

    self.generator.save(os.path.join(output_model_path, f'generator_model_{GENERATE_SQUARE}_res_{GENERATE_RES}_seed_{SEED_SIZE}_epochs_{EPOCHS}_batchSize_{BATCH_SIZE}_bufferSize_{BUFFER_SIZE}.npy'))
    self.discriminator.save(os.path.join(output_model_path, f'discriminator_model_{GENERATE_SQUARE}_res_{GENERATE_RES}_seed_{SEED_SIZE}_epochs_{EPOCHS}_batchSize_{BATCH_SIZE}_bufferSize_{BUFFER_SIZE}.npy'))

  def load_models(self):
    output_model_path = os.path.join(OUTPUT_IMAGE_PATH, self.data_name + "_Models")
    self.generator = load_model(os.path.join(output_model_path, f'generator_model_{GENERATE_SQUARE}_res_{GENERATE_RES}_seed_{SEED_SIZE}_epochs_{EPOCHS}_batchSize_{BATCH_SIZE}_bufferSize_{BUFFER_SIZE}.npy'))
    self.discriminator = load_model(os.path.join(output_model_path, f'discriminator_model_{GENERATE_SQUARE}_res_{GENERATE_RES}_seed_{SEED_SIZE}_epochs_{EPOCHS}_batchSize_{BATCH_SIZE}_bufferSize_{BUFFER_SIZE}.npy'))

  def save_images(self, count, noise):
    image_array = np.full(( 
        PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
        PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3), 
        255, dtype=np.uint8)
    
    print(image_array.shape)
    
    generated_images = self.generator.predict(noise)

    generated_images = 0.5 * generated_images + 0.5

    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
          
          r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
          c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN

          # print("Rows ", r+GENERATE_SQUARE)
          # print("Cols ", c + GENERATE_SQUARE)
          image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] = generated_images[image_count] * 255
          image_count += 1

            
    output_path = os.path.join(DATA_PATH,'output')
    if not os.path.exists(output_path):
      os.makedirs(output_path)
    
    filename = os.path.join(output_path,f"train-{count}.png")
    im = Image.fromarray(image_array)
    im.save(filename)
  
  def run(self, new_model, save_img_checkpoint, save_model_checkpoint):
    if new_model:
      #Build generator and discriminator 
      self.generator = self.build_generator()
      self.discriminator = self.build_discriminator()
    else:
      #Load previous model
      self.load_models()

    #Compiling G and D
    self.compile_models()

    print('Starting training...')
    self.train(self.training_dataset, self.training_labels, EPOCHS, save_img_checkpoint, save_model_checkpoint)

    print('Saving generator model...')
    self.save_models()




#### Running the Model

In [5]:
if __name__ == "__main__":
  can = CAN()
  can.run(new_model = True, save_img_checkpoint = 100, save_model_checkpoint = 500)

  0%|          | 0/4 [00:07<?, ?it/s]


Starting training...
tf.Tensor(115.97182, shape=(), dtype=float32)
tf.Tensor(9.936165, shape=(), dtype=float32)


KeyboardInterrupt: ignored